In [11]:
import pandas as pd

# Load the CSV file
file_path = 'data/student_data_unprocessed.csv'
df = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
df.head()


,Name,Email,EMBA,Resume,Evaluating Potential Markets/Choosing Target Market,Developing Pricing Model,Customer Decision Journey,Pilot Project Design and Implementation,Investment Pitch Deck,Hiring Plan,...,Digital Tech track ranking,Digital Health track ranking,Deep Tech track ranking,Life Sciences track ranking,EFL Track Preference,Deep Tech Companies Preferences,Digital Tech Companies Preferences,Life Sciences Companies Preferences,Digital Health Companies Preferences,Contacted Startup?
0,"Aggarwal, Dipak",da3457@stern.nyu.edu,No,Dipak_Aggarwal - Resume.pdf,7.0,6.0,4.0,6.0,5.0,1.0,...,4,1,2,3,NaN,"Enigma Aerospace,Navaflex,Wright Electric,REEV...","DataHive,Hoox,Safe YOU | Impact Innovations In...","Libera Bio,CartaBio,Olfera,BlueWhale Bio,Celli...","YCare,Solvuu,Stimvia,Picture Health,Torpedo Th...",NaN
1,"Ahmad, Omar",omar.ahmad@stern.nyu.edu,No,Omar Ahmad Resume.pdf,4.0,5.0,2.0,1.0,4.0,4.0,...,2,1,3,4,Have a flight on Meeting Day 1 of Deep Tech - ...,"Enigma Aerospace,Navaflex,Propitious Technolog...","Lore Machine,VoicePlug,Spectronn,PLATMA","Rejuvenation Technologies,BreakBio Corp,SEED T...","SOAP Health,Atmo Biosciences,nSight Surgical,L...",NaN
2,"Akers, Travis",tda2296@stern.nyu.edu,No,Travis Akers 9.18.24.pdf,7.0,4.0,4.0,5.0,4.0,3.0,...,4,3,1,2,I have a conflicting class on October 18th and...,"Cyanotype Bio,Enigma Aerospace,Robochef,Avol,W...","Elm AI,Lore Machine,dataspan.ai,PLATMA,syd.lif...","BlueWhale Bio,BreakBio Corp,iLoF,SEED Therapeu...","LUCID,YCare,Banquet Health,Fort Health,nSight ...",NaN
3,"Ali Darwish, Samir",sna8444@stern.nyu.edu,No,Resume - Samir Ali Darwish.pdf,4.0,4.0,2.0,2.0,5.0,5.0,...,1,2,3,4,It was tough reading the life sciences startup...,"ATOMICS,Avol,Enigma Aerospace,Frost Methane,Cy...","SpruceID,Revolv3,VoicePlug,Trust Science,CTGT,...","iLoF,Sensible Biotechnologies,Cloverleaf Bio,R...","YCare,nSight Surgical,MiCare Path,Siftwell Ana...",NaN
4,"Anifa Saitbai, Abraar",aa10948@stern.nyu.edu,No,Abraar's Resume final.docx.pdf,4.0,3.0,5.0,5.0,6.0,5.0,...,1,3,2,4,NaN,"Verne,RISE Robotics,Sunchem,Page Technologies,...","Spectronn,Smabbler,NightVision Security,Trust ...","Ceramedix,Cloverleaf Bio,IDP Pharma,Koi Biothe...","PathKeeper Surgical,Banquet Health,ExoDiscover...","The founder of Spectronn reached out to me, an..."


In [12]:
# Step 1: Create the required columns
output_df = pd.DataFrame()

# List of startups in current cohort - MUST match the order they appear in startups_data sheet
startup_names = [
'BreakBio Corp', 'CartaBio', 'Wright Electric', 'DataHive', '42Genetics', 'SOAP Health', 'Cyanotype Bio', 'Free Form Fibers', 'MabSwitch', 
'Hoox', 'Elm AI', 'Godot', 'ZayaAI', 'Telness Tech', 'Smabbler','CAREier', 'PathKeeper Surgical', 'GrayMatters Health', 'SyntheX',
'Helix Earth', 'AdviNOW Medical', 'Sensible Biotechnologies', 'Modulate Bio', 'Torpedo Therapeutics', 'IDP Pharma', 'Werewool', 'Rasayana Therapeutics',
'Cellinfinity Bio', 'dataspan.ai', 'Libera Bio', 'RealNose.ai', 'Simulacra SDS', 'Evvy', 'Atmo Biosciences', 'Avol', 'LATERAL.systems', 'Coastar Therapeutics', 
'Rejuvenation Technologies', 'Ceramedix', 'MiCare Path', 'Picture Health', 'BlueWhale Bio', 'Siftwell Analytics', 'DoriNano', 'Script Biosciences', 
'Relyion Energy', 'Olfera', 'RISE Robotics', 'Lore Machine', 'Fermbox Bio', 'SEED Therapeutics', 'Revolv3', 'Nest Genomics', 'Frost Methane', 
'Sama Therapeutics', 'CTGT', 'REEV', 'Serentrix', 'NavaFlex', 'ByteGenie', 'Koi Biotherapeutics', 'CLIKA', 'Cloverleaf Bio', 'Sunchem', 'Robochef', 
'OLLANG', 'Luxsonic', 'Tessel Bio', 'SpruceID', 'Plantibodies', 'PLATMA', 'ExoDiscovery', 'Suyana Climate Insurance', 'Stimvia', 'Apricity Health', 'iLoF', 
'ATOMICS', 'Banquet Health', 'nSight Surgical', 'Banyan Infrastructure', 'Vitro3D', 'Solvuu', 'The Rubic', 'Agilis Commerce', 
'Trust Science', 'syd.life', 'Enigma Aerospace', 'Page Technologies', 'Safe YOU | Impact Innovations Institute', 'Verne', 'geCKo Materials', 'Schnell LiFi Germany',
'LUCID', 'Merlin Biotech', 'Propitious Technologies', 'Youverify', 'WORMSENSING', 'VoicePlug', 'Fenix Commerce'
]


# Extract student name
output_df['student name'] = df['Name']

# Convert 'yes' and 'no' to 1 and 0 in the 'EMBA' column
output_df['emba'] = df['EMBA'].map({'Yes': 1, 'No': 0})

# Track rankings columns (Deep Tech, Digital Tech, Life Sciences, Digital Health)
# These are scaled to a range of 0.25 to 1 based on the rank value
def scale_ranking(rank):
    return 1.25 - (0.25 * rank)

output_df['Deep tech rank'] = df['Deep Tech track ranking'].apply(scale_ranking)
output_df['Digital tech rank'] = df['Digital Tech track ranking'].apply(scale_ranking)
output_df['Life sciences rank'] = df['Life Sciences track ranking'].apply(scale_ranking)
output_df['Digital health rank'] = df['Digital Health track ranking'].apply(scale_ranking)

# Step 2: Normalize skill values by dividing by 7 (assuming certain columns correspond to skills)
# Columns containing skill values (manual inspection required based on column names)
skill_columns = [
    'Evaluating Potential Markets/Choosing Target Market',
    'Developing Pricing Model',
    'Customer Decision Journey',
    'Pilot Project Design and Implementation',
    'Investment Pitch Deck',
    'Hiring Plan',
    'Marketing Plan & Execution',
    'Business Development & Sales',
    'Scaling Operations'
]
for col in skill_columns:
    output_df[col] = df[col] / 7

# Step 3: Generate startup preference columns
# Assuming preference columns are under the following structure:
preference_columns = [
    'Deep Tech Companies Preferences',
    'Digital Tech Companies Preferences',
    'Life Sciences Companies Preferences',
    'Digital Health Companies Preferences'
]

# Create columns for each startup
for startup in startup_names:
    output_df[startup] = 0.1  # Default value

# Function to assign weights based on preference rank
def assign_weights(pref_list, no_of_companies):
    weights = {}
    for idx, company in enumerate(pref_list):
        weight = (no_of_companies - idx) * (1 / no_of_companies)
        weights[company.strip()] = weight
    return weights

# Fill the startup preferences with scaled weights based on the student's preference
for i, row in df.iterrows():
    for col in preference_columns:
        if row[col]:
            company_list = row[col].split(',')
            company_weights = assign_weights(company_list, len(company_list))
            for company, weight in company_weights.items():
                if company in output_df.columns:
                    output_df.at[i, company] = weight

# Pulling matched students from the algo
student_names_remove = ['Zhang, Hui', 'Hayden, Nicholas', 'Zhai, Danni', 'Anifa Saitbai, Abraar']
filtered_df = output_df[~output_df['student name'].isin(student_names_remove)]

# Round all floating point values to 2 decimal places
filtered_df = filtered_df.round(2)

# Save the updated output to CSV
output_file_rounded = 'data/processed_student_data.csv'
filtered_df.to_csv(output_file_rounded, index=False)

output_file_rounded

/var/folders/n6/wdpx2fcx4ds7sr7yq7pd__lc0000gn/T/ipykernel_64821/2103552076.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[startup] = 0.1  # Default value
/var/folders/n6/wdpx2fcx4ds7sr7yq7pd__lc0000gn/T/ipykernel_64821/2103552076.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[startup] = 0.1  # Default value
/var/folders/n6/wdpx2fcx4ds7sr7yq7pd__lc0000gn/T/ipykernel_64821/2103552076.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

'data/processed_student_data.csv'

In [13]:
output_df.head

<bound method NDFrame.head of               student name  emba  Deep tech rank  Digital tech rank  \
0          Aggarwal, Dipak     0            0.75               0.25   
1              Ahmad, Omar     0            0.50               0.75   
2            Akers, Travis     0            1.00               0.25   
3       Ali Darwish, Samir     0            0.50               1.00   
4    Anifa Saitbai, Abraar     0            0.75               1.00   
..                     ...   ...             ...                ...   
97            Zeng, Chutao     0            0.50               1.00   
98             Zhai, Danni     0            0.25               0.75   
99            Zhang, Alcor     0            0.25               0.50   
100             Zhang, Hui     0            0.25               0.50   
101           Zheng, Ximan     0            0.50               0.75   

     Life sciences rank  Digital health rank  \
0                  0.50                 1.00   
1                  0.